In [360]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from IPython.core.interactiveshell import InteractiveShell
from datetime import timedelta
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df_before = pd.read_csv('CR_FC_PREP_0522.csv')
df_before['POD'] = pd.to_datetime(df_before['POD'])

df_before.head()
df_before.tail()
df_before.info()
print(f'Dataset size: {df_before.shape}')

# Y is the target
Y = df_before.pop('RATE') 
X = df_before

#normalise
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


,POD,RATE
6370,2022-12-27,4121.43
6371,2022-12-28,4142.86
6372,2022-12-29,4164.29
6373,2022-12-30,4185.71
6374,2022-12-31,4207.14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     6375 non-null   datetime64[ns]
 1   RATE    6375 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.7 KB
Dataset size: (6375, 2)


In [361]:
# After 2023 data to compare accuracy of forecast
df_after = pd.read_csv('CR_FC_PREP_23.csv')
df_after['POD'] = pd.to_datetime(df_after['POD'])

# df_after.set_index('POD',inplace=True)

df_after.head()
df_after.tail()
df_after.info()
print(f'Dataset size: {df_after.shape}')


actual_X = df_after.drop("RATE", axis=1)
actual_y = df_after["RATE"]

,POD,RATE
0,2023-01-01,4228.57
1,2023-01-02,4250.00
2,2023-01-03,2800.00
3,2023-01-04,2800.00
4,2023-01-05,2800.00


,POD,RATE
47,2023-02-17,1500.0
48,2023-02-18,1500.0
49,2023-02-19,1500.0
50,2023-02-20,1500.0
51,2023-02-21,1500.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     52 non-null     datetime64[ns]
 1   RATE    52 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 960.0 bytes
Dataset size: (52, 2)


In [362]:
def findRF(X_train, X_test, y_train, y_test):
    # Build the random forest model
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)

    # Predict the test data
    y_pred = rf_model.predict(X_test)

    # Calculate the root mean squared error
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Calculate the R-squared value
    r2 = r2_score(y_test, y_pred)

    # Calculate the accuracy score in percentage
    mean = y_test.mean()
    accuracy = (1 - rmse/mean) * 100

    print("Root Mean Squared Error:", round(rmse, 3))
    print("R-squared Value:", round(r2, 3))
    print("Accuracy Score:", round(accuracy, 2), "%")

    return y_pred

In [363]:
n_periods = 52
y_pred_last_52 = findRF(X_train, X_test, y_train, y_test)[-n_periods:]
print(y_pred_last_52)

Root Mean Squared Error: 105.684
R-squared Value: 0.997
Accuracy Score: 93.23 %
[3982.7251 1500.     1200.     2200.      409.4854  900.     1500.
  900.     4492.4257  909.     9328.65    795.8331  900.     8143.2373
 8150.     8425.104  4298.3636  900.3334  912.7025  534.69    957.0262
  590.49    761.42   7795.5402  520.      604.4    1629.9714 1532.25
 1620.      900.     4583.832   448.      567.2438 1649.54   2200.
  809.25    100.     1153.7387 1200.      858.1884  900.     3694.3517
  900.      430.     9436.6643  595.      430.     1619.5428 2107.1612
  934.9084  447.96   1116.0021]


In [364]:
def findACC(y_pred_last_52, actual_y):
    # Calculate the accuracy of the model using mean absolute error and R-squared
   mae = mean_absolute_error(actual_y, y_pred_last_52)
   r2 = r2_score(actual_y, y_pred_last_52)

   print(f"Mean absolute error: {mae:.2f}")
   print(f"R-squared value: {r2:.2f}")

In [365]:
findACC(y_pred_last_52, actual_y)

Mean absolute error: 1947.98
R-squared value: -9.23


In [366]:
# # Plot the forecast with current dataset
# plt.figure(figsize=(30, 8))
# plt.plot(df_bf_indexed)
# plt.plot(forecast_df, color='darkgreen')
# plt.title("Final Forecast of COST_RATE")
# plt.show()